In [1]:
import stanza
import pandas as pd
import numpy as np
import re
import tensorflow as tf
import tqdm as notebook_tqdm

In [2]:
nlp = stanza.Pipeline(lang="id",use_gpu=True)

2024-05-20 15:40:05 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-05-20 15:40:06 INFO: Loading these models for language: id (Indonesian):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| lemma     | gsd     |
| depparse  | gsd     |

2024-05-20 15:40:06 WARNING: GPU requested, but is not available!
2024-05-20 15:40:06 INFO: Using device: cpu
2024-05-20 15:40:06 INFO: Loading: tokenize
2024-05-20 15:40:06 INFO: Loading: mwt
2024-05-20 15:40:06 INFO: Loading: pos
2024-05-20 15:40:06 INFO: Loading: lemma
2024-05-20 15:40:06 INFO: Loading: depparse
2024-05-20 15:40:06 INFO: Done loading processors!


In [3]:
doc = nlp("Budi makan nasi")
print("token:")
doc.sentences[0].print_tokens()

token:
<Token id=1;words=[<Word id=1;text=Budi;lemma=budi;upos=PROPN;xpos=NSD;head=2;deprel=nsubj>]>
<Token id=2;words=[<Word id=2;text=makan;lemma=makan;upos=VERB;xpos=VSA;feats=Mood=Ind|Voice=Act;head=0;deprel=root>]>
<Token id=3;words=[<Word id=3;text=nasi;lemma=nasi;upos=NOUN;xpos=NSD;feats=Number=Sing;head=2;deprel=obj>]>


In [4]:
# Import dataframe
import pandas as pd
dev_file = 'E:\\Suwija Putra\\datasetTextualEntailment\\SNLI Indo Bersih\\SNLI_Indo_val.jsonl'
df_dev = pd.read_json(dev_file, lines=True) 
df_dev

,annotator_labels,label emas,kalimat1,kalimat2
0,"['netral', 'entailment', 'netral', 'netral', ...",netral,Dua wanita berpelukan sambil memegang paket un...,Para suster memeluk selamat tinggal sambil mem...
1,"['entailment', 'entailment', 'entailment', 'e...",keterlibatan,Dua wanita berpelukan sambil memegang paket un...,Dua wanita memegang paket.
2,"['kontradiksi', 'kontradiksi', 'kontradiksi',...",kontradiksi,Dua wanita berpelukan sambil memegang paket un...,Orang-orang berkelahi di luar toko makanan.
3,"['entailment', 'entailment', 'entailment', 'e...",keterlibatan,"Dua anak kecil berbaju biru, satu dengan nomor...",Dua anak dengan kaus bernomor mencuci tangan m...
4,"['netral', 'netral', 'netral', 'entailment', ...",netral,"Dua anak kecil berbaju biru, satu dengan nomor...",Dua anak di ballgame mencuci tangan mereka.
...,...,...,...,...
9836,"['entailment', 'entailment', 'entailment', 'e...",keterlibatan,Seorang peserta motocross berseragam telah mel...,Seseorang mengendarai sepeda motor dengan peno...
9837,"['entailment', 'entailment', 'entailment', 'e...",keterlibatan,Seorang gadis kecil mengenakan jaket merah mud...,Korsel bergerak.
9838,"['kontradiksi', 'kontradiksi', 'kontradiksi',...",kontradiksi,Seorang gadis kecil mengenakan jaket merah mud...,Gadis itu bergerak dengan kecepatan cahaya.
9839,"['entailment', 'entailment', 'entailment', 'en...",keterlibatan,Seorang gadis muda dengan pita biru dan merah ...,Orang-orang di air mancur


In [5]:
df_dev.shape

(9841, 4)

In [6]:
import string
df_dev_kalimat1 = df_dev
df_dev_kalimat1 = df_dev_kalimat1['kalimat1']
# Ambil 10 data pertama
df_dev_kalimat2 = df_dev
# Ambil kalimat kedua saja sebagai contoh
df_dev_kalimat2 = df_dev_kalimat2['kalimat2']

df_label = df_dev
df_label = df_label['label emas']
# Ubah ke lowercase
def remove_punctuation(text):
    text = text.lower()
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)
df_dev_kalimat1 = df_dev_kalimat1.apply(remove_punctuation)
df_dev_kalimat2 = df_dev_kalimat2.apply(remove_punctuation)

In [7]:
list_dependencies_2 = list()
for i in df_dev_kalimat2:
    # print(i)
    doc = nlp(i)
    dependencies_list = doc.sentences[0].dependencies
    list_dependencies = list()
    for x in range(len(dependencies_list)):
        # print(dependencies_list[x][0].text)
        # print(dependencies_list[x][2].text)
        list_dependencies.append([dependencies_list[x][0].text,dependencies_list[x][2].text])
    # print(list_dependencies)
    list_dependencies_2.append(list_dependencies)
# list_dependencies_all

list_dependencies_1 = list()
for i in df_dev_kalimat1:
    # print(i)
    doc = nlp(i)
    dependencies_list = doc.sentences[0].dependencies
    list_dependencies = list()
    for x in range(len(dependencies_list)):
        # print(dependencies_list[x][0].text)
        # print(dependencies_list[x][2].text)
        list_dependencies.append([dependencies_list[x][0].text,dependencies_list[x][2].text])
    # print(list_dependencies)
    list_dependencies_1.append(list_dependencies)

In [8]:
filtered_list1 = [[sublist for sublist in subsublist if 'ROOT' not in sublist] for subsublist in list_dependencies_1]
filtered_list2 = [[sublist for sublist in subsublist if 'ROOT' not in sublist] for subsublist in list_dependencies_2]
print(df_dev_kalimat1[0])
print(filtered_list1[0])

dua wanita berpelukan sambil memegang paket untuk pergi
[['wanita', 'dua'], ['berpelukan', 'wanita'], ['memegang', 'sambil'], ['berpelukan', 'memegang'], ['memegang', 'paket'], ['pergi', 'untuk'], ['memegang', 'pergi']]


In [9]:
# Head
head_1 = list()
for item in filtered_list1:
    head_item = list()
    for i in item:
        head_item.append(i[0])
    head_1.append(head_item)
# print(head_1)

dependecy_1 = list()
for item in filtered_list1:
    dependecy_item = list()
    for i in item:
        dependecy_item.append(i[1])
    dependecy_1.append(dependecy_item)
# print(dependecy_1)

head_2 = list()
for item in filtered_list2:
    head_item = list()
    for i in item:
        head_item.append(i[0])
    head_2.append(head_item)
# print(head_1)

dependecy_2 = list()
for item in filtered_list2:
    dependecy_item = list()
    for i in item:
        dependecy_item.append(i[1])
    dependecy_2.append(dependecy_item)

In [10]:
head_dependecy = pd.DataFrame({'head1':head_1,'dependecy1':dependecy_1,'head2':head_2,'dependecy2':dependecy_2})
head_dependecy = pd.concat([head_dependecy,df_label],axis=1)
head_dependecy

,head1,dependecy1,head2,dependecy2,label emas
0,"[wanita, berpelukan, memegang, berpelukan, mem...","[dua, wanita, sambil, memegang, paket, untuk, ...","[suster, memeluk, memeluk, memeluk, memegang, ...","[para, suster, selamat, tinggal, sambil, memeg...",netral
1,"[wanita, berpelukan, memegang, berpelukan, mem...","[dua, wanita, sambil, memegang, paket, untuk, ...","[wanita, memegang, memegang]","[dua, wanita, paket]",keterlibatan
2,"[wanita, berpelukan, memegang, berpelukan, mem...","[dua, wanita, sambil, memegang, paket, untuk, ...","[berkelahi, luar, berkelahi, luar, toko]","[orangorang, di, luar, toko, makanan]",kontradiksi
3,"[anak, berdiri, anak, anak, berbaju, biru, nom...","[dua, anak, kecil, berbaju, biru, satu, dengan...","[anak, bernomor, kaus, anak, bernomor, mencuci...","[dua, anak, dengan, kaus, mencuci, tangan, mer...",keterlibatan
4,"[anak, berdiri, anak, anak, berbaju, biru, nom...","[dua, anak, kecil, berbaju, biru, satu, dengan...","[anak, mencuci, ballgame, anak, mencuci, tangan]","[dua, anak, di, ballgame, tangan, mereka]",netral
...,...,...,...,...,...
9836,"[peserta, melewati, peserta, peserta, melewati...","[seorang, peserta, motocross, berseragam, tela...","[mengendarai, mengendarai, sepeda, penonton, m...","[seseorang, sepeda, motor, dengan, penonton]",keterlibatan
9837,"[gadis, mengenakan, gadis, mengenakan, jaket, ...","[seorang, gadis, kecil, jaket, merah, muda, se...",[bergerak],[korsel],keterlibatan
9838,"[gadis, mengenakan, gadis, mengenakan, jaket, ...","[seorang, gadis, kecil, jaket, merah, muda, se...","[bergerak, gadis, kecepatan, bergerak, kecepatan]","[gadis, itu, dengan, kecepatan, cahaya]",kontradiksi
9839,"[gadis, mengarungi, gadis, pita, gadis, pita, ...","[seorang, gadis, muda, dengan, pita, biru, dan...","[air, orangorang, air]","[di, air, mancur]",keterlibatan


In [11]:
head_dependecy = pd.DataFrame({'head1':head_1,'dependecy1':dependecy_1,'head2':head_2,'dependecy2':dependecy_2})
head_dependecy = pd.concat([head_dependecy,df_label],axis=1)
head_dependecy

,head1,dependecy1,head2,dependecy2,label emas
0,"[wanita, berpelukan, memegang, berpelukan, mem...","[dua, wanita, sambil, memegang, paket, untuk, ...","[suster, memeluk, memeluk, memeluk, memegang, ...","[para, suster, selamat, tinggal, sambil, memeg...",netral
1,"[wanita, berpelukan, memegang, berpelukan, mem...","[dua, wanita, sambil, memegang, paket, untuk, ...","[wanita, memegang, memegang]","[dua, wanita, paket]",keterlibatan
2,"[wanita, berpelukan, memegang, berpelukan, mem...","[dua, wanita, sambil, memegang, paket, untuk, ...","[berkelahi, luar, berkelahi, luar, toko]","[orangorang, di, luar, toko, makanan]",kontradiksi
3,"[anak, berdiri, anak, anak, berbaju, biru, nom...","[dua, anak, kecil, berbaju, biru, satu, dengan...","[anak, bernomor, kaus, anak, bernomor, mencuci...","[dua, anak, dengan, kaus, mencuci, tangan, mer...",keterlibatan
4,"[anak, berdiri, anak, anak, berbaju, biru, nom...","[dua, anak, kecil, berbaju, biru, satu, dengan...","[anak, mencuci, ballgame, anak, mencuci, tangan]","[dua, anak, di, ballgame, tangan, mereka]",netral
...,...,...,...,...,...
9836,"[peserta, melewati, peserta, peserta, melewati...","[seorang, peserta, motocross, berseragam, tela...","[mengendarai, mengendarai, sepeda, penonton, m...","[seseorang, sepeda, motor, dengan, penonton]",keterlibatan
9837,"[gadis, mengenakan, gadis, mengenakan, jaket, ...","[seorang, gadis, kecil, jaket, merah, muda, se...",[bergerak],[korsel],keterlibatan
9838,"[gadis, mengenakan, gadis, mengenakan, jaket, ...","[seorang, gadis, kecil, jaket, merah, muda, se...","[bergerak, gadis, kecepatan, bergerak, kecepatan]","[gadis, itu, dengan, kecepatan, cahaya]",kontradiksi
9839,"[gadis, mengarungi, gadis, pita, gadis, pita, ...","[seorang, gadis, muda, dengan, pita, biru, dan...","[air, orangorang, air]","[di, air, mancur]",keterlibatan


In [12]:
head_dependecy['head1'] = head_dependecy['head1'].apply(lambda x: ' '.join(x))
head_dependecy['dependecy1'] = head_dependecy['dependecy1'].apply(lambda x: ' '.join(x))
head_dependecy['head2'] = head_dependecy['head2'].apply(lambda x: ' '.join(x))
head_dependecy['dependecy2'] = head_dependecy['dependecy2'].apply(lambda x: ' '.join(x))
pd.set_option('display.max_colwidth',1)
head_dependecy

,head1,dependecy1,head2,dependecy2,label emas
0,wanita berpelukan memegang berpelukan memegang pergi memegang,dua wanita sambil memegang paket untuk pergi,suster memeluk memeluk memeluk memegang memeluk memegang pergi memegang makan pergi makan,para suster selamat tinggal sambil memegang paket untuk pergi setelah makan siang,netral
1,wanita berpelukan memegang berpelukan memegang pergi memegang,dua wanita sambil memegang paket untuk pergi,wanita memegang memegang,dua wanita paket,keterlibatan
2,wanita berpelukan memegang berpelukan memegang pergi memegang,dua wanita sambil memegang paket untuk pergi,berkelahi luar berkelahi luar toko,orangorang di luar toko makanan,kontradiksi
3,anak berdiri anak anak berbaju biru nomor berbaju nomor satu 9 nomor nomor nomor tangga berdiri tangga kamar tangga kamar mencuci berdiri mencuci wastafel mencuci,dua anak kecil berbaju biru satu dengan nomor 9 dan satu dengan nomor 2 di tangga kayu di kamar mandi dan mencuci tangan di wastafel,anak bernomor kaus anak bernomor mencuci tangan,dua anak dengan kaus mencuci tangan mereka,keterlibatan
4,anak berdiri anak anak berbaju biru nomor berbaju nomor satu 9 nomor nomor nomor tangga berdiri tangga kamar tangga kamar mencuci berdiri mencuci wastafel mencuci,dua anak kecil berbaju biru satu dengan nomor 9 dan satu dengan nomor 2 di tangga kayu di kamar mandi dan mencuci tangan di wastafel,anak mencuci ballgame anak mencuci tangan,dua anak di ballgame tangan mereka,netral
...,...,...,...,...,...
9836,peserta melewati peserta peserta melewati lompatan melewati lintasan lompatan mengangkangi melewati mengangkangi sepeda motor udara sepeda pemuda pemuda mengangkangi melihat pemuda,seorang peserta motocross berseragam telah sebuah lompatan di lintasan dan mengangkangi sepeda motor nya di udara dengan beberapa pemuda yang melihat,mengendarai mengendarai sepeda penonton mengendarai,seseorang sepeda motor dengan penonton,keterlibatan
9837,gadis mengenakan gadis mengenakan jaket jaket mengendarai mengenakan mengendarai,seorang gadis kecil jaket merah muda sedang mengendarai korsel,bergerak,korsel,keterlibatan
9838,gadis mengenakan gadis mengenakan jaket jaket mengendarai mengenakan mengendarai,seorang gadis kecil jaket merah muda sedang mengendarai korsel,bergerak gadis kecepatan bergerak kecepatan,gadis itu dengan kecepatan cahaya,kontradiksi
9839,gadis mengarungi gadis pita gadis pita merah biru merah kepang pita kepang atasan pita wanita wanita atasan tshirt wanita tshirt rok tshirt rok mengarungi air mancur,seorang gadis muda dengan pita biru dan merah muda di kepang nya tanpa atasan dan seorang wanita dengan tshirt putih dan rok zebra air mancur umum,air orangorang air,di air mancur,keterlibatan


In [13]:
# save the file
head_dependecy.to_csv('E:\\Suwija Putra\\datasetTextualEntailment\\indonli\\dependency_val_snli_new.csv', index=False) # change the name of the file